In [1]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from pathlib import Path
import torch

In [ ]:
import os
os.chdir("..")
print("Current working directory:", os.getcwd())

Current working directory: /home/smokehappiest/cnn-age-recognize
Current directory: /home/smokehappiest/cnn-age-recognize


In [9]:
class AgeDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = Path(f"{self.img_dir}/{row['image_name']}")
        image = Image.open(img_path).convert('RGB')
        age = float(row['age'])
        
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(age, dtype=torch.float32)

In [11]:
from torchvision import transforms

basic_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_ds = AgeDataset('datasets/splits/train.csv', 'datasets/splits/train', transform=basic_tfms)
val_ds   = AgeDataset('datasets/splits/val.csv', 'datasets/splits/validate', transform=basic_tfms)

from torch.utils.data import DataLoader
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=32, shuffle=False)

In [12]:
import torch.nn as nn
from torchvision import models

class ResNetAgeRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        num_ftrs = base.fc.in_features
        base.fc = nn.Identity()
        self.features = base
        self.head = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.features(x)
        return self.head(x)

In [13]:
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # On my pc correct CUDA )
model = ResNetAgeRegressor().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)

def mae(pred, target):
    return torch.mean(torch.abs(pred - target))

best_val = float('inf')

for epoch in range(10):  # начнём с 10 эпох
    model.train()
    train_loss = 0
    for imgs, ages in tqdm(train_dl, desc=f"Epoch {epoch+1}/10 [Train]"):
        imgs, ages = imgs.to(device), ages.to(device).unsqueeze(1)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, ages)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)

    model.eval()
    val_loss, val_mae = 0, 0
    with torch.no_grad():
        for imgs, ages in tqdm(val_dl, desc=f"Epoch {epoch+1}/10 [Valid]"):
            imgs, ages = imgs.to(device), ages.to(device).unsqueeze(1)
            preds = model(imgs)
            loss = criterion(preds, ages)
            val_loss += loss.item() * imgs.size(0)
            val_mae += mae(preds, ages).item() * imgs.size(0)

    train_loss /= len(train_ds)
    val_loss /= len(val_ds)
    val_mae /= len(val_ds)
    print(f"Epoch {epoch+1}: Train MSE={train_loss:.3f}, Val MSE={val_loss:.3f}, Val MAE={val_mae:.3f}")

    if val_mae < best_val:
        best_val = val_mae
        torch.save(model.state_dict(), 'best_resnet50_age.pth')
        print("✅ Saved new best model")

Epoch 1/10 [Valid]: 100%|██████████| 196/196 [00:10<00:00, 18.94it/s]


Epoch 1: Train MSE=176.811, Val MSE=82.369, Val MAE=6.640
✅ Saved new best model


Epoch 2/10 [Valid]: 100%|██████████| 196/196 [00:10<00:00, 19.24it/s]


Epoch 2: Train MSE=68.137, Val MSE=86.611, Val MAE=6.997


Epoch 3/10 [Valid]: 100%|██████████| 196/196 [00:10<00:00, 19.11it/s]


...

Epoch 8: Train MSE=19.017, Val MSE=66.373, Val MAE=5.891
✅ Saved new best model


Epoch 9/10 [Valid]: 100%|██████████| 196/196 [00:10<00:00, 18.60it/s]


Epoch 9: Train MSE=16.474, Val MSE=65.396, Val MAE=5.819
✅ Saved new best model


Epoch 10/10 [Valid]: 100%|██████████| 196/196 [00:10<00:00, 18.58it/s]


Epoch 10: Train MSE=15.718, Val MSE=64.455, Val MAE=5.754
✅ Saved new best model
